In [0]:
# !pip uninstall -y kaggle
# !pip install --upgrade pip
# !pip install kaggle==1.5.6
# !kaggle -v

In [0]:
!mkdir -p ~/.kaggle

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d airplane2230/apparel-image-dataset-2

 94% 233M/249M [00:05<00:00, 42.1MB/s]
100% 249M/249M [00:05<00:00, 45.4MB/s]


In [0]:
! unzip -q -n /content/apparel-image-dataset-2.zip  
import pandas as pd
import numpy as np

In [0]:
train_set = pd.read_csv('/content/train.csv')
val_set=pd.read_csv('/content/val.csv')
test_set=pd.read_csv('/content/test.csv')

In [92]:
val_set=pd.concat([val_set,test_set])

train_set=train_set.replace(1.0, pd.Series(train_set.columns, train_set.columns))
train_set=train_set.replace(0.0,'')

val_set=val_set.replace(1.0, pd.Series(val_set.columns, val_set.columns))
val_set=val_set.replace(0.0,'')

val_set

,Unnamed: 0,image,black,blue,brown,green,red,white,dress,shirt,pants,shorts,shoes
0,0,./clothes_dataset\black_shirt\caf19003d77246a8...,black,,,,,,,shirt,,,
1,1,./clothes_dataset\blue_pants\7e42face2c655f1bf...,,blue,,,,,,,pants,,
2,2,./clothes_dataset\red_dress\d46069bf0d2c1cc6bb...,,,,,red,,dress,,,,
3,3,./clothes_dataset\blue_dress\fea68beee2e52c48d...,,blue,,,,,dress,,,,
4,4,./clothes_dataset\white_shoes\b378e85b811cfef4...,,,,,,white,,,,,shoes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3411,3411,./clothes_dataset\black_shirt\d76faa497083f1f6...,black,,,,,,,shirt,,,
3412,3412,./clothes_dataset\blue_pants\691dacb1274608390...,,blue,,,,,,,pants,,
3413,3413,./clothes_dataset\blue_pants\9b5ac8ce3de1d5219...,,blue,,,,,,,pants,,
3414,3414,./clothes_dataset\black_pants\7db76af76fe19a01...,black,,,,,,,,pants,,


In [93]:
train_set['labels'] = train_set.iloc[:,2:13].apply(lambda row:' '.join(row.values.astype(str)), axis=1)
train_set['labels'] = train_set['labels'].replace('\s+', ' ', regex=True)
train_set['labels'] = train_set['labels'].replace('^\s*\s*$', '', regex=True)

val_set['labels'] = val_set.iloc[:,2:13].apply(lambda row:' '.join(row.values.astype(str)), axis=1)
val_set['labels'] = val_set['labels'].replace('\s+', ' ', regex=True)
val_set['labels'] = val_set['labels'].replace('^\s*\s*$', '', regex=True)
val_set

,Unnamed: 0,image,black,blue,brown,green,red,white,dress,shirt,pants,shorts,shoes,labels
0,0,./clothes_dataset\black_shirt\caf19003d77246a8...,black,,,,,,,shirt,,,,black shirt
1,1,./clothes_dataset\blue_pants\7e42face2c655f1bf...,,blue,,,,,,,pants,,,blue pants
2,2,./clothes_dataset\red_dress\d46069bf0d2c1cc6bb...,,,,,red,,dress,,,,,red dress
3,3,./clothes_dataset\blue_dress\fea68beee2e52c48d...,,blue,,,,,dress,,,,,blue dress
4,4,./clothes_dataset\white_shoes\b378e85b811cfef4...,,,,,,white,,,,,shoes,white shoes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3411,3411,./clothes_dataset\black_shirt\d76faa497083f1f6...,black,,,,,,,shirt,,,,black shirt
3412,3412,./clothes_dataset\blue_pants\691dacb1274608390...,,blue,,,,,,,pants,,,blue pants
3413,3413,./clothes_dataset\blue_pants\9b5ac8ce3de1d5219...,,blue,,,,,,,pants,,,blue pants
3414,3414,./clothes_dataset\black_pants\7db76af76fe19a01...,black,,,,,,,,pants,,,black pants
